In [1]:
import sys
import os
from pathlib import Path

from IPython.display import Markdown
from openai import OpenAI
from dotenv import dotenv_values

# Add the parent directory to sys.path so Python can find the Utils module
sys.path.append(os.path.abspath(".."))

from Utils import (
    get_guidance_and_feedback,
    get_questions_and_answers,
    create_content,
)

# Get required environment variables
config = dotenv_values("../.env")
open_api_key = config.get("OPENAI_API_KEY", None)
tutor_agent_id = config.get("TUTOR_AGENT_ID", None)

# Check if the environment variables are set
if not (tutor_agent_id and open_api_key):
    print("Need TUTOR_AGENT_ID and OPEN_API_KEY in the .env file")
    sys.exit(1)

# Create the OpenAI client
client = OpenAI(api_key=config["OPENAI_API_KEY"])

Generate the content for our prompt message to our bot.  Our prompt must look like the input examples we provided to our assistant in its instructions.  For this toy example we are using JSON files of question data and student answers as a store.

In [2]:
q_path = Path('../Data/question_store.json')
a_path = Path('../Data/student_answers.json')
questions, answers = get_questions_and_answers(q_path, a_path)

Having loaded the question and answer data we can now create input content.

In [3]:
question_id = "Q7_43"
student_id = "student_3"

question = questions[question_id]
student_answer = answers[question_id][student_id]

content = create_content(question, student_answer)

Now we start a new conversation thread and send our content as a messages to our assistant.  This can take some time to run.

In [7]:
thread, messages, run = get_guidance_and_feedback(
    client, content, tutor_agent_id, temperature=0.1, thread_id=None, show_run=True)

**Bot Message**:


Your response to the exam question shows some understanding of the impact of external factors on the hydrogen atom, but it contains significant inaccuracies and omissions that need to be addressed to fully meet the requirements of the question.

**Feedback on Your Answer:**

1. **Misunderstanding of Energy Modification**: You mentioned that the energy modification is due to the absorption of a photon. This is incorrect in the context of the question. The energy modification in this scenario is due to the interaction of the electron's magnetic moment with an external magnetic field, not photon absorption.

2. **Hamiltonian Modification**: Your answer does not address how the Hamiltonian operator is modified in the presence of a magnetic field. The correct approach is to include a term that accounts for the interaction between the electron's magnetic moment and the magnetic field, specifically the Zeeman term.

3. **Schrodinger Equation**: You did not provide the Schrodinger equation for the hydrogen atom in the absence of an external magnetic field as requested. The correct form involves the Hamiltonian operator that includes only the kinetic and Coulomb potential terms.

4. **Energy Levels in a Magnetic Field**: You did not discuss how the energy levels of the hydrogen atom are affected by the external magnetic field. The presence of the magnetic field leads to a splitting of the energy levels, a phenomenon known as the Zeeman effect.

**Correct Concepts and Guidance:**

- **Energy Modification**: The energy of the hydrogen atom in a magnetic field includes the original kinetic and potential energy terms plus an additional term for the magnetic interaction. This can be expressed as:
  $$
  E = E_0 + \frac{eB_zL_z}{2m_e}
  $$
  where $E_0$ is the energy without the magnetic field.

- **Hamiltonian Operator**: The Hamiltonian in the presence of a magnetic field is:
  $$
  \hat{H} = \hat{H_0} + \frac{eB_z}{2m_e} \hat{L}_z
  $$
  This includes the original Hamiltonian $\hat{H_0}$ and an additional term that represents the interaction of the electron's orbital angular momentum with the magnetic field.

- **Schrodinger Equation**: In the absence of a magnetic field, the Schrodinger equation is:
  $$
  \hat{H_0} \psi = E \psi
  $$
  where $\hat{H_0}$ includes the kinetic and Coulomb potential terms.

- **Energy Levels with Magnetic Field**: In the presence of a magnetic field, the energy levels are split according to the magnetic quantum number $m_l$, reflecting the Zeeman effect:
  $$
  E = E_0 + \beta_B m_l B_z
  $$
  where $\beta_B = \frac{e\hbar}{2m_e}$.

**Suggested Sections for Review**:
- Review the sections on the Hamiltonian for a hydrogen atom and the effects of a magnetic field. Specifically, look at the sections discussing the Zeeman effect and how magnetic fields interact with atomic structures.
- Chapter 7, particularly around the development of the Hamiltonian and the Schrodinger equation for systems in external fields.

By revisiting these concepts and correcting the misunderstandings in your answer, you'll gain a better grasp of how external fields influence quantum systems like the hydrogen atom.



**Run object attributes**:
- status: completed
- temperature: 0.1
- tools: [FileSearchTool(type='file_search')]
- usage: Usage(completion_tokens=696, prompt_tokens=3869, total_tokens=4565)


Now we have started the thread, we can continue if we want the chat with the bot.

In [5]:
follow_up = "Which section of the textbook explains the splitting of energy levels when the atom is in an external magnetic field?"
thread, messages, run = get_guidance_and_feedback(
    client, follow_up, tutor_agent_id, temperature=0.1, thread_id=thread, show_run=True)

**Bot Message**:


The splitting of energy levels of a hydrogen atom when placed in an external magnetic field is discussed in Chapter 7, Section 7.4 of the textbook "Quantum Chemistry" by Donald A. McQuarrie. This section explains how the energy levels are split into sublevels, leading to the splitting of spectral lines, a phenomenon known as the Zeeman effect. For a detailed explanation, you can refer to this section in your textbook [0].
[0] None from QUANTUM_CHEMISTRY_SECOND_EDITION.pdf


**Run object attributes**:
- status: completed
- temperature: 0.1
- tools: [FileSearchTool(type='file_search')]
- usage: Usage(completion_tokens=119, prompt_tokens=24551, total_tokens=24670)
